In [3]:

## SOURCE CODE AND NOTEBOOK -TINGYU  -IEL MEMBER (PROJECT TEAM MEMBER)







import os
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
%matplotlib inline
import missingno as msno
np.random.seed(19)
from matplotlib.pyplot import figure
data_directory = '/work2/05067/nagyz/austin_energy/data/'
filepaths = [os.path.join(data_directory,f) for f in os.listdir(data_directory) if f.endswith('.csv')]
from datetime import datetime 
from Graphs import load_data,load_graph_kwh,yearly_graph_kwh,load_graph_count,monthly_graph_kwh,daily_graph_kwh
from Graphs import load_graph_kwh_and_count,yearly_graph_kwh_and_count,monthly_graph_kwh_and_count,daily_graph_kwh_and_count
from agg_and_heatmap_functions import length,prav3,prav2,prav1,dinostorm,plot_aggraph,heatmap,finalheatmap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, SGDRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_dir = './data/'
export_dir = os.path.join(data_dir, '_4_exports_UT')
id_list = pd.read_csv('temporal_percentage1.csv')
id_list.drop('Unnamed: 0',axis=1,inplace=True)
weather_daily=pd.read_csv('Austin_weather_daily.csv')
weather_daily['time']=pd.to_datetime(weather_daily['time'],utc=True)
weather_daily1=weather_daily.set_index('time',drop=True)
weather_austin_daily=weather_daily1['tavg']
w1=weather_austin_daily['2017-01-01 00:00:00+00:00':'2021-03-01 00:00:00+00:00']
w1

time
2017-01-01 00:00:00+00:00    17.0
2017-01-02 00:00:00+00:00    17.4
2017-01-03 00:00:00+00:00    15.2
2017-01-04 00:00:00+00:00     5.9
2017-01-05 00:00:00+00:00     8.6
                             ... 
2021-02-25 00:00:00+00:00    13.8
2021-02-26 00:00:00+00:00    12.0
2021-02-27 00:00:00+00:00    19.5
2021-02-28 00:00:00+00:00    21.7
2021-03-01 00:00:00+00:00    11.7
Name: tavg, Length: 1521, dtype: float64

In [23]:
id_list['buildingType'].unique()
res_type = ['SINGLE FAMILY', 'MULTIFAMILY',  'FOURPLEX', 
            'DUPLEX', 'CONDOS', 'CONDO (STACKED)', 
            '1 FAM DWELLING, ACCESSORY DWELLING UNIT',
            'TOWNHOMES','MOHO SINGLE PP', 
            '1 FAM DWELLING, GARAGE APARTMENT', 'MOHO DOUBLE PP',
            'MOHO SINGLE REAL', 'MOHO DOUBLE REAL', 'TRIPLEX', 
            '1 FAM DWELLING, MOHO DOUBLE REAL',
           '1 FAM DWELLING, MOHO SINGLE REAL']

commercial_type = ['SM OFFICE CONDO', 'RESTAURANT',
       'OFFICE MED 10-35', 'OFFICE LG >35000', 'OFFICE (SMALL)',
       'LG OFFICE CONDO', 'REGIONAL SHOP CT', 'FAST FOOD REST',
       'SM STORE <10K SF','CONVENIENCE STOR',
       'COMMERCIAL IMPROVED', 'COMMERCIAL SPACE CONDOS',
       'OFF HI-RISE >= 6']

other_type = [x for x in id_list['buildingType'].unique() if x not in res_type and x not in  commercial_type]

In [24]:
def create_winterstorm_data(point):
    example_ts=pd.read_csv('/work2/05067/nagyz/austin_energy/data/_4_exports_UT/UT_sID_'+str(point)+'_dates_2017.01.01_through_2021.10.01.csv')
    example_ts.index = pd.to_datetime(example_ts.index)
    start_time='2017-01-01 00:00:00+00:00'
    end_time='2021-03-01 00:00:00+00:00'
    example_ts=example_ts[start_time:end_time]
    df = pd.DataFrame()
    for year in range(2017, 2020):
        year = str(year)
        row_i_1to3 = w1[f'{year}-01':f'{year}-03'].argmin()
        month_1to3 = w1.index[row_i_1to3].month
        index_string1 = f'{year}-{month_1to3}'
        df = pd.concat([df, example_ts[index_string1:index_string1]])
        
        row_i_4to6 = w1[f'{year}-04':f'{year}-06'].argmin()
        month_4to6 = w1.index[row_i_4to6].month
        index_string2 = f'{year}-{month_4to6}'
        df = pd.concat([df, example_ts[index_string2:index_string2]])
        
        row_i_7to9 = w1[f'{year}-07':f'{year}-09'].argmin()
        month_7to9 = w1.index[row_i_7to9].month
        index_string3 = f'{year}-{month_7to9}'
        df = pd.concat([df, example_ts[index_string3:index_string3]])
        
        row_i_10to12 = w1[f'{year}-10':f'{year}-12'].argmin()
        month_10to12 = w1.index[row_i_10to12].month
        index_string4 = f'{year}-{month_10to12}'
        df = pd.concat([df, example_ts[index_string4:index_string4]])
        
        
    df = pd.concat([df, example_ts['2021-01-01':'2021-01-31']])

    return df

In [25]:
weather_daily=pd.read_csv('Austin_weather_daily.csv')
weather_daily['time']=pd.to_datetime(weather_daily['time'],utc=True)
weather_daily1=weather_daily.set_index('time',drop=True)
weather_austin_daily=weather_daily1['tavg']
cool_points=weather_austin_daily[weather_austin_daily<15]['2017-01-01 00:00:00+00:00':'2021-01-31 00:00:00+00:00']
winter_index=weather_austin_daily['2021-01-31 00:00:00+00:00':'2021-02-28 00:00:00+00:00'].index
s=pd.Series(index = cool_points.index)+pd.Series(index = winter_index)
s.index

DatetimeIndex(['2017-01-04 00:00:00+00:00', '2017-01-05 00:00:00+00:00',
               '2017-01-06 00:00:00+00:00', '2017-01-07 00:00:00+00:00',
               '2017-01-08 00:00:00+00:00', '2017-01-09 00:00:00+00:00',
               '2017-01-14 00:00:00+00:00', '2017-01-17 00:00:00+00:00',
               '2017-01-18 00:00:00+00:00', '2017-01-19 00:00:00+00:00',
               ...
               '2021-02-19 00:00:00+00:00', '2021-02-20 00:00:00+00:00',
               '2021-02-21 00:00:00+00:00', '2021-02-22 00:00:00+00:00',
               '2021-02-23 00:00:00+00:00', '2021-02-24 00:00:00+00:00',
               '2021-02-25 00:00:00+00:00', '2021-02-26 00:00:00+00:00',
               '2021-02-27 00:00:00+00:00', '2021-02-28 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=392, freq=None)

In [29]:
freq, freq_long_name ='D', 'daily'
check_timedelta = pd.Timedelta(1, freq)

        # residential
res_type_id_list = id_list[id_list['buildingType'].isin(res_type)].copy()
res_type_id_list['weight'] = res_type_id_list['squareFt'] / res_type_id_list['squareFt'].sum()
daily_date_range = pd.date_range('20210301', '20210731', freq=freq, tz = 'UTC')
res_type_id_list_ts = pd.Series(index = daily_date_range)

for day in tqdm(daily_date_range):
    energy_data = pd.DataFrame(columns = ['squareFt', 'energyUsage'])
    for i, row in res_type_id_list.iterrows():
        square_ft = row['squareFt']
        point = int(row['Service Point'])

        example_ts=pd.read_csv('/work2/05067/nagyz/austin_energy/data/_4_exports_UT/UT_sID_'+str(point)+'_dates_2017.01.01_through_2021.10.01.csv')
        example_ts.index = pd.to_datetime(example_ts.index)
        daily_data = example_ts[day:day + check_timedelta][:-1]
        energy_data.loc[point] = [square_ft, daily_data.iloc[:, 0].mean()]

    weigheted_data = (energy_data['energyUsage'] * energy_data['squareFt'] / energy_data['squareFt'].sum()).sum()
    res_type_id_list_ts[day] = weigheted_data


100%|██████████| 153/153 [4:40:55<00:00, 110.16s/it]  


In [28]:
res_type_id_list_ts

2021-03-01 00:00:00+00:00    1181.713679
2021-03-02 00:00:00+00:00    1199.621811
2021-03-03 00:00:00+00:00    1160.971100
2021-03-04 00:00:00+00:00    1043.311550
2021-03-05 00:00:00+00:00    1006.897375
                                ...     
2021-07-27 00:00:00+00:00            NaN
2021-07-28 00:00:00+00:00            NaN
2021-07-29 00:00:00+00:00            NaN
2021-07-30 00:00:00+00:00            NaN
2021-07-31 00:00:00+00:00            NaN
Freq: D, Length: 153, dtype: float64

In [29]:
res_type_id_list_ts.to_csv('resi_covid_.csv')

In [6]:
freq, freq_long_name ='D', 'daily'
check_timedelta = pd.Timedelta(1, freq)
check_qa_len = len(pd.date_range(pd.Timestamp('2017-01-01T12'), pd.Timestamp('2017-01-01T12') + check_timedelta, freq = '15min')[:-1]) / 2
    
    # residential
other_type_id_list = id_list[id_list['buildingType'].isin(other_type)].copy()
other_type_id_list['weight'] = other_type_id_list['squareFt'] / other_type_id_list['squareFt'].sum()
daily_date_range = pd.date_range('20210101', '20210301', freq=freq, tz = 'UTC')
other_type_id_list_ts = pd.Series(index = daily_date_range)
for day in tqdm(daily_date_range):
    energy_data = pd.DataFrame(columns = ['squareFt', 'Count'])
    for i, row in other_type_id_list.iterrows():
        square_ft = row['squareFt']
        point = int(row['Service Point'])
            
        example_ts=pd.read_csv('/work2/05067/nagyz/austin_energy/data/_4_exports_UT/UT_sID_'+str(point)+'_dates_2017.01.01_through_2021.10.01.csv')
        example_ts.index = pd.to_datetime(example_ts.index)
        daily_data = example_ts[day:day + check_timedelta][:-1]
        if len(daily_data) > check_qa_len: # if the value contains like 50% below, then we do not count for that
            energy_data.loc[point] = [square_ft, daily_data.iloc[:, 2].mean()]
            
    weigheted_data = (energy_data['Count'] * energy_data['squareFt'] / energy_data['squareFt'].sum()).sum()
    #weigheted_count= (energy_data['count'] * energy_data['squareFt'] / energy_data['squareFt'].sum()).sum()
    other_type_id_list_ts[day] = weigheted_data

100%|██████████| 60/60 [29:02<00:00, 29.05s/it]


In [12]:
other_type_id_list_ts.to_csv('other_2021_part.csv')

In [22]:
base_line=make_pipeline(RandomForestRegressor())